In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display

In [3]:
# Load data
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [5]:
# Clean title function
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [7]:
movies["clean_title"] = movies["title"].apply(clean_title)
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [9]:
# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["clean_title"])

In [11]:
# Search function
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results

In [13]:
# Recommendation function
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]
    

In [15]:
# Interactive search without widgets
def interactive_search():
    while True:
        title = input("Enter a movie title (or 'quit' to exit): ")
        if title.lower() == 'quit':
            break
        if len(title) > 3:
            try:
                results = search(title)
                if not results.empty:
                    print("\nSearch results:")
                    display(results)
                    
                    movie_id = results.iloc[0]["movieId"]
                    recommendations = find_similar_movies(movie_id)
                    
                    print("\nRecommendations based on", results.iloc[0]["title"])
                    display(recommendations)
                else:
                    print("No results found. Try a different movie title.")
            except Exception as e:
                print(f"Error: {e}")
        else:
            print("Please enter at least 4 characters.")
        print("\n" + "-"*50 + "\n")

# Run the interactive search
interactive_search()

Enter a movie title (or 'quit' to exit):  Avengers



Search results:


,movieId,title,genres,clean_title
34536,145676,3 Avengers (1964),(no genres listed),3 Avengers 1964
2063,2153,"Avengers, The (1998)",Action|Adventure,Avengers The 1998
17067,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
40636,159920,Shaolin Avengers (1994),Action,Shaolin Avengers 1994
45394,170297,Ultimate Avengers 2 (2006),Action|Animation|Sci-Fi,Ultimate Avengers 2 2006



Recommendations based on 3 Avengers (1964)


,score,title,genres



--------------------------------------------------



Enter a movie title (or 'quit' to exit):  quit


In [17]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [19]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [21]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [23]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [25]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [27]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [29]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [31]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [33]:
rec_percentages

,similar,all
movieId,,
89745,1.000000,0.040459
58559,0.573393,0.148256
59315,0.530649,0.054931
79132,0.519715,0.132987
2571,0.496687,0.247010
...,...,...
47610,0.103545,0.022770
780,0.103380,0.054723
88744,0.103048,0.010383


In [35]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [37]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [39]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


In [41]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [ ]:
# Interactive search function (non-widget version)
def interactive_movie_search():
    print("Movie Recommendation System")
    print("Enter a movie title to get recommendations (type 'quit' to exit)")
    
    while True:
        title = input("\nEnter movie title: ")
        
        if title.lower() == 'quit':
            print("Goodbye!")
            break
            
        if len(title) > 3:  # Reduced threshold for better usability
            try:
                # Search for the movie
                results = search(title)
                
                if not results.empty:
                    print(f"\nTop search results for '{title}':")
                    for i, row in results.iterrows():
                        print(f"{i+1}. {row['title']} ({row['genres']})")
                    
                    # Get recommendations based on the first result
                    movie_id = results.iloc[0]["movieId"]
                    recommendations = find_similar_movies(movie_id)
                    
                    print(f"\nRecommendations similar to '{results.iloc[0]['title']}':")
                    display(recommendations)
                else:
                    print("No results found. Please try a different movie title.")
                    
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Please enter at least 4 characters.")

# Run the interactive search
interactive_movie_search()

Movie Recommendation System
Enter a movie title to get recommendations (type 'quit' to exit)



Enter movie title:  Avengers



Top search results for 'Avengers':
34537. 3 Avengers (1964) ((no genres listed))
2064. Avengers, The (1998) (Action|Adventure)
17068. Avengers, The (2012) (Action|Adventure|Sci-Fi|IMAX)
40637. Shaolin Avengers (1994) (Action)
45395. Ultimate Avengers 2 (2006) (Action|Animation|Sci-Fi)

Recommendations similar to '3 Avengers (1964)':


,score,title,genres



Enter movie title:  quit


Goodbye!
